In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import scipy.io as sio
import scipy.sparse as sps

from ampute import ampute_mcar, ampute_mar, compute_mar_probas

import joblib
import os # to play sound at the end of the execution

In [3]:
dataset_names = ['RCV1', 'WebACE', 'classic3', 'classic4', 'cstr', 'ng20', 'reviews', 'sports']
for dname in dataset_names:
    m = sio.loadmat(f'data/{dname}.mat')
    print(f'{dname!r}: ', [*filter(lambda x: not x.startswith('__'),[*m])])

'RCV1':  ['ans', 'gnd', 'fea', 'm', 'n', 'I', 'j', 'nz', 'i', 'x', 'X1', 'sumX']
'WebACE':  ['gnd', 'fea']
'classic3':  ['A', 'ts', 'ms', 'labels', 'cK']
'classic4':  ['labels', 'mat', 'fea']
'cstr':  ['gnd', 'fea']
'ng20':  ['labels', 'mat', 'fea']
'reviews':  ['dtm', 'classid']
'sports':  ['dtm', 'classid']


In [10]:
keys = {
    'cstr':  ['gnd', 'fea'],
    'WebACE':  ['gnd', 'fea'],
    'classic3':  ['labels', 'A'],
    'classic4':  ['labels', 'mat'],
    'reviews':  ['classid', 'dtm'],
#     'sports':  ['classid', 'dtm'],
#     'RCV1':  ['gnd', 'fea'],
#     'ng20':  ['labels', 'mat'],
}

In [11]:
na_props = [.1, .2, .5, .7]

In [12]:
dataset = 'RCV1'
na_prop = 0.1
ampute_method = 'mar'
init_method = 'rand'

In [13]:
mat = sio.loadmat(f'data/{dataset}.mat')

In [14]:
X = mat[keys[dataset][1]]
y = mat[keys[dataset][0]]
type(y)

scipy.sparse.csc.csc_matrix

In [ ]:
for ampute_method in ('mar',):
    print("#"*15, ampute_method, "#"*15)
    for dataset in keys:
        print("*"*10, dataset, "*"*10)
        mat = sio.loadmat(f'data/{dataset}.mat')
        X = mat[keys[dataset][1]]
        y = mat[keys[dataset][0]]
        if isinstance(X, sps.csr_matrix) or isinstance(X, sps.csc_matrix):
            X = X.toarray()
        if ampute_method == 'mar':
            missing_probas, _ = compute_mar_probas(X)
        
        for na_prop in na_props:
            if ampute_method == 'mcar':
                X_missing, r_nan, c_nan = ampute_mcar(X, na_prop)
            elif ampute_method == 'mar':
                X_missing, r_nan, c_nan = ampute_mar(X, na_prop, M_proba=missing_probas)
            joblib.dump({
                'r_nan': r_nan,
                'c_nan': c_nan,
            }, f'data/out/nan_idx-{dataset}-{ampute_method}-{na_prop}.joblib', compress=3)
    
os.system('echo -e "\07" && echo -e "\007"')

############### mar ###############
********** RCV1 **********
